# Part 14

## # Dataset

In [91]:
import pandas as pd
import numpy as np

In [185]:
df = pd.read_csv('../datasets/petr4_treinamento.csv')
df.shape, df.columns

((1245, 7),
 Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object'))

In [186]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2013-01-02,19.990000,20.209999,19.690001,19.690001,18.086271,30182600.0
1,2013-01-03,19.809999,20.400000,19.700001,20.400000,18.738441,30552600.0
2,2013-01-04,20.330000,20.620001,20.170000,20.430000,18.766001,36141000.0
3,2013-01-07,20.480000,20.670000,19.950001,20.080000,18.444506,28069600.0
4,2013-01-08,20.110001,20.230000,19.459999,19.500000,17.911745,29091300.0


In [187]:
def _isnull(df):
    for c in df.columns:
        cnt = df[pd.isnull(df[c])].shape[0]
        if cnt != 0:
            print(f'# {c}: {cnt} rows')

In [188]:
_isnull(df)

# Open: 3 rows
# High: 3 rows
# Low: 3 rows
# Close: 3 rows
# Adj Close: 3 rows
# Volume: 3 rows


In [189]:
df.dropna(axis=0, inplace=True)
df.reset_index(inplace=True, drop=True)
df.shape

(1242, 7)

In [190]:
DAYS_BEFORE = 90

In [191]:
df.shape

(1242, 7)

In [192]:
df['x'] = np.nan
df['x'].iloc[90:] = np.zeros(shape=(1152,))
df.iloc[87:92]

/home/hbrandao/.local/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,Date,Open,High,Low,Close,Adj Close,Volume,x
87,2013-05-14,19.530001,19.680000,19.250000,19.549999,18.298189,30643700.0,NaN
88,2013-05-15,19.500000,19.639999,19.330000,19.570000,18.316908,21869300.0,NaN
89,2013-05-16,19.520000,20.080000,19.459999,19.750000,18.485382,25067200.0,NaN
90,2013-05-17,19.910000,20.049999,19.790001,19.889999,18.616419,19040100.0,0.0
91,2013-05-20,19.910000,20.250000,19.559999,20.200001,18.906569,30571700.0,0.0


In [208]:
def daysbefore(days_before, col):
    feature = []
    target = []
    for i in range(days_before, df.shape[0]):
        slice_start = i - days_before
        slice_stop = i
        feature.append(df[col].iloc[slice_start:slice_stop].values)
        target.append(df[col].iloc[i])
    _ = pd.DataFrame(data=open_feature)
    _['target'] = target
    return _

In [210]:
df_open = daysbefore(90, 'Open')
df_open.shape

(1242, 9)

In [212]:
df_open.head()

,0,1,2,3,4,5,6,7,8,9,...,81,82,83,84,85,86,87,88,89,target
0,19.990000,19.809999,20.330000,20.480000,20.110001,19.639999,19.770000,19.850000,20.010000,20.010000,...,20.01,20.400000,20.750000,20.250000,20.160000,19.809999,19.530001,19.500000,19.520000,19.910000
1,19.809999,20.330000,20.480000,20.110001,19.639999,19.770000,19.850000,20.010000,20.010000,19.889999,...,20.40,20.750000,20.250000,20.160000,19.809999,19.530001,19.500000,19.520000,19.910000,19.910000
2,20.330000,20.480000,20.110001,19.639999,19.770000,19.850000,20.010000,20.010000,19.889999,19.860001,...,20.75,20.250000,20.160000,19.809999,19.530001,19.500000,19.520000,19.910000,19.910000,20.190001
3,20.480000,20.110001,19.639999,19.770000,19.850000,20.010000,20.010000,19.889999,19.860001,19.799999,...,20.25,20.160000,19.809999,19.530001,19.500000,19.520000,19.910000,19.910000,20.190001,20.270000
4,20.110001,19.639999,19.770000,19.850000,20.010000,20.010000,19.889999,19.860001,19.799999,19.570000,...,20.16,19.809999,19.530001,19.500000,19.520000,19.910000,19.910000,20.190001,20.270000,19.549999


## # Model

## # Evaluation